In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, median_absolute_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

In [2]:
path = 'https://raw.githubusercontent.com/Codecademy/datasets/master/streeteasy/queens.csv'
df = pd.read_csv(path)
df.head(3)

,rental_id,rent,bedrooms,bathrooms,size_sqft,min_to_subway,floor,building_age_yrs,no_fee,has_roofdeck,has_washer_dryer,has_doorman,has_elevator,has_dishwasher,has_patio,has_gym,neighborhood,borough
0,10234,3000,3.0,1,1000,4,1.0,106,0,0,0,0,0,0,0,0,Astoria,Queens
1,10524,1950,1.0,1,950,1,3.0,83,0,0,0,0,0,0,0,0,Forest Hills,Queens
2,5465,3500,1.0,1,996,9,7.0,3,1,0,0,0,0,0,0,0,Astoria,Queens


In [3]:
y = df['rent']
X = df[['bedrooms', 'bathrooms', 'size_sqft', 'min_to_subway', 'floor', 'building_age_yrs', 'no_fee']]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.8, random_state=2019)

# XgBoost

* [docs](https://xgboost.readthedocs.io/en/latest/tutorials/model.html)
* [paper](https://arxiv.org/pdf/1603.02754.pdf)

In [5]:
from xgboost import XGBRegressor

In [15]:
model = XGBRegressor(random_state=2019, max_depth=10, n_estimators=1000)

In [16]:
model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bytree=1, gamma=0, importance_type='gain',
             learning_rate=0.1, max_delta_step=0, max_depth=10,
             min_child_weight=1, missing=None, n_estimators=1000, n_jobs=1,
             nthread=None, objective='reg:linear', random_state=2019,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=True, subsample=1)

In [17]:
y_pred = model.predict(X_train)

In [18]:
mean_absolute_error(y_train, y_pred)

0.1420700669954609

In [19]:
y_pred2 = model.predict(X_test)

In [20]:
mean_absolute_error(y_test, y_pred2)

254.51304660373265

## Hyperparam Optimisation

In [21]:
scorer = make_scorer(mean_absolute_error)

In [28]:
grid = GridSearchCV(model, param_grid={'n_estimators':[1000, 1200,2000]}, scoring=scorer, cv=5)

In [29]:
grid.fit(X_train, y_train)

/Users/philippk/anaconda3/envs/packt/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/philippk/anaconda3/envs/packt/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/philippk/anaconda3/envs/packt/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/philippk/anaconda3/envs/packt/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/philippk/anaconda3/envs/packt/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated a

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bytree=1,
                                    gamma=0, importance_type='gain',
                                    learning_rate=0.1, max_delta_step=0,
                                    max_depth=10, min_child_weight=1,
                                    missing=None, n_estimators=1000, n_jobs=1,
                                    nthread=None, objective='reg:linear',
                                    random_state=2019, reg_alpha=0,
                                    reg_lambda=1, scale_pos_weight=1, seed=None,
                                    silent=True, subsample=1),
             iid='warn', n_jobs=None,
             param_grid={'n_estimators': [1000, 1200, 2000]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=make_scorer(mean_absolute_e

In [30]:
grid.best_score_

343.18827265734114

In [31]:
pd.DataFrame(grid.cv_results_)[['params', 'mean_test_score']]

,params,mean_test_score
0,{'n_estimators': 1000},343.185545
1,{'n_estimators': 1200},343.186090
2,{'n_estimators': 2000},343.188273
